# Carga Data INGRESOS X ROL

Creación del DataFrame asociado a los Ingresos por rol

Rev: 29-10-2020

In [1]:
import os
import pandas as pd
import numpy as np
from pyarrow import feather
from tqdm import tqdm

from src.data import cleandata

In [2]:
path_raw = "../data/raw/pjud" 
archivos = os.listdir(path_raw) 
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
# Creacion dataframe con los datos de Ingresos por Rol Penal de los años 2015 a 2019
dataframes = []

for archivo in archivos:
    if archivo.find("Ingresos por Rol Penal") != -1:
        df = pd.read_csv(f"{path_raw}/{archivo}", sep = ";", encoding = 'cp850', low_memory = True)
        dataframes.append(df) 
        
df_ingresos_rol = pd.concat(dataframes, axis = 0)
df_ingresos_rol.dtypes

N°                int64
COD. CORTE        int64
CORTE            object
COD. TRIBUNAL     int64
TRIBUNAL         object
RIT              object
TIPO CAUSA       object
FECHA INGRESO    object
MES INGRESO      object
AÑO INGRESO       int64
dtype: object

In [4]:
# Transformamos variables float64 a int16

df_ingresos_rol['COD. CORTE'] = df_ingresos_rol['COD. CORTE'].fillna(0).astype(np.int16)
df_ingresos_rol['COD. TRIBUNAL'] = df_ingresos_rol['COD. TRIBUNAL'].fillna(0).astype(np.int16)
df_ingresos_rol['AÑO INGRESO'] = df_ingresos_rol['AÑO INGRESO'].fillna(0).astype(np.int16)
df_ingresos_rol.drop(['N°'], axis = 'columns', inplace = True)

In [5]:
df_ingresos_rol['FECHA INGRESO'] = df_ingresos_rol['FECHA INGRESO'].progress_apply(cleandata.convierte_fecha)

100%|██████████| 2980977/2980977 [00:05<00:00, 557521.57it/s]


In [6]:
# Elimino espacios en las columnas tipo objetos

df_ingresos_rol = df_ingresos_rol.progress_apply(cleandata.elimina_espacios, axis=0)

100%|██████████| 9/9 [00:14<00:00,  1.64s/it]


In [7]:
# Elimino tildes de las columnas object

cols = df_ingresos_rol.select_dtypes(include = ["object"]).columns
df_ingresos_rol[cols] = df_ingresos_rol[cols].progress_apply(cleandata.elimina_tilde)

100%|██████████| 5/5 [00:38<00:00,  7.62s/it]


In [11]:
# Transformamos en variables categoricas

df_ingresos_rol['CORTE'] = df_ingresos_rol['CORTE'].astype('category')

Dejo solo causas Ordinarias

In [12]:
df_ingresos_rol['TIPO CAUSA'].unique()

[Ordinaria, Militar, Extradición]
Categories (3, object): [Ordinaria, Militar, Extradición]

In [13]:
tipo_causa = df_ingresos_rol[df_ingresos_rol['TIPO CAUSA']!='Ordinaria']

In [14]:
df_ingresos_rol.drop(tipo_causa.index, axis=0, inplace=True)

In [15]:
# Reset el index para realizar feather

df_ingresos_rol.reset_index(inplace = True)
df_ingresos_rol

,level_0,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,MES INGRESO,AÑO INGRESO
0,0,0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,1-2018,Ordinaria,2018-01-02,ene-18,2018
1,1,1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,2-2018,Ordinaria,2018-01-03,ene-18,2018
2,2,2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,3-2018,Ordinaria,2018-01-03,ene-18,2018
3,3,3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,4-2018,Ordinaria,2018-01-03,ene-18,2018
4,4,4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,5-2018,Ordinaria,2018-01-03,ene-18,2018
...,...,...,...,...,...,...,...,...,...,...,...
2980478,2980972,583609,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,167-2016,Ordinaria,2016-12-21,dic-16,2016
2980479,2980973,583610,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,168-2016,Ordinaria,2016-12-22,dic-16,2016
2980480,2980974,583611,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,169-2016,Ordinaria,2016-12-22,dic-16,2016
2980481,2980975,583612,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,170-2016,Ordinaria,2016-12-27,dic-16,2016


In [16]:
# Guardamos dataset como archivo feather
path_interim = "../data/interim/pjud"
os.makedirs(path_interim, exist_ok = True) 

df_ingresos_rol.to_feather(f'{path_interim}/IngresosRol_feather')